In [4]:
from transformers import AutoModel
from numpy.linalg import norm

cos_sim = lambda a,b: (a @ b.T) / (norm(a)*norm(b))
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-zh', trust_remote_code=True) # trust_remote_code is needed to use the encode method
embeddings = model.encode(['How is the weather today?', '今天天气怎么样?'])
print(cos_sim(embeddings[0], embeddings[1]))

/data/clipx/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [16]:
emoji_des_fn = "emoji_table.txt"
emoji_table = [l.strip().split("\t") for l in open(emoji_des_fn).readlines()]

In [18]:
emoji_table[1]

['Aerial Tramway', 'U+1F6A1']

In [21]:
emoji_des = {}
for d,k in emoji_table:
    if k in emoji_des:
        emoji_des[k].append(d)
    else:
        emoji_des[k] = [d]
emoji_des = {k:", ".join(v) for k,v in emoji_des.items()}

In [22]:
emoji_des

{'U+1F39F': 'Admission Tickets',
 'U+1F6A1': 'Aerial Tramway, Cable Car, Ropeway, Gondola',
 'U+2708': 'Airplane, Plane, Aeroplane, Flight, Flying',
 'U+23F0': 'Alarm Clock, Clock, Alarm',
 'U+2697': 'Alembic',
 'U+1F47E': 'Alien Monster, Space Invader, Video Game Monster',
 'U+1F691': 'Ambulance',
 'U+1F3C8': 'American Football, Football, Gridiron, Superbowl',
 'U+1F3FA': 'Amphora, Jar, Vase',
 'U+2693': 'Anchor, Fisherman, Admiralty Pattern Anchor',
 'U+1F4A2': 'Anger Symbol, Anger Sign, Punch',
 'U+1F620': 'Angry Face, Angry, Grumpy Face',
 'U+1F627': 'Anguished Face, Pained Face',
 'U+1F41C': 'Ant, Black Ant, Bullant',
 'U+1F4F6': 'Antenna With Bars, Reception Bars, Signal Strength, Stairs',
 'U+1F504': 'Anticlockwise Downwards and Upwards Open Circle Arrows, Refresh Icon, Reverse Arrows',
 'U+2652': 'Aquarius',
 'U+2648': 'Aries',
 'U+2935': 'Arrow Pointing Rightwards Then Curving Downwards, Curved Down Arrow',
 'U+2934': 'Arrow Pointing Rightwards Then Curving Upwards',
 'U+1F69B

In [27]:
emoji_embeddings = {k:model.encode(v) for k,v in emoji_des.items()}

In [28]:
len(emoji_embeddings)

1012

In [29]:
import pickle
# save emoji_embeddings
pickle.dump(emoji_embeddings, open('emoji_embeddings.pkl', 'wb'))

# load emoji_embeddings
emoji_embeddings = pickle.load(open('emoji_embeddings.pkl', 'rb'))

In [35]:
emoji_embeddings = pickle.load(open('emoji_embeddings.pkl', 'rb'))

In [38]:
nomred_emoji_embeddings = {k:v/norm(v) for k,v in emoji_embeddings.items()}
pickle.dump(nomred_emoji_embeddings, open('emoji_normed_embeddings.pkl', 'wb'))
emoji_embeddings = pickle.load(open('emoji_normed_embeddings.pkl', 'rb'))

In [51]:
emoji_code_list = np.array(list(emoji_embeddings.keys()))
def decode_emoji(emoji_code):
    return chr(int(emoji_code[2:], 16))
emoji_list = np.array([decode_emoji(emoji_code) for emoji_code in emoji_code_list])

In [44]:
import numpy as np
emoji_Fs = np.stack(list(emoji_embeddings.values()))

In [85]:
# dump emoji_Fs use numpy

np.save("emoji.npy", {"emoji_features": emoji_Fs, "emoji_list": emoji_list, "emoji_code_list": emoji_code_list})
    


emoji = np.load('emoji.npy', allow_pickle=True).item()
emoji_features = emoji['emoji_features']
emoji_list

np.load('emoji.npy', allow_pickle=True)

In [2]:
import numpy as np

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


In [75]:
def query_emoji(description, n_candidates=5):
    query = model.encode(description)
    query = query/norm(query)
    scores = emoji_Fs @ query
    inds = np.argpartition(-scores, min(n_candidates, len(emoji_list)-1))[:n_candidates]
    return emoji_list[inds].tolist(), emoji_code_list[inds].tolist(), scores[inds].tolist()

In [84]:
query_emoji("抽烟")

(['🌫', '🚬', '⛰', '🥓', '🕯'],
 ['U+1F32B', 'U+1F6AC', 'U+26F0', 'U+1F953', 'U+1F56F'],
 [0.34354108572006226,
  0.4129222631454468,
  0.3472767472267151,
  0.36155179142951965,
  0.3562557101249695])

In [82]:
np.argpartition([1,2],1)[:10]

array([0, 1])